In [297]:
import pandas as pd
import datetime
import math

In [298]:
train_df = pd.read_csv('data/train.csv', sep=";", na_values="?")
test_df = pd.read_csv('data/test.csv', sep=";", na_values="?")

train_df.head()

,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,ratio of urban inhabitants,average salary,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '96,unemploymant rate '95,unemploymant rate '96,withdrawal total,household (avg),credit total,First trans date
0,5314,1787,930705,96396,12,8033,-1,30,weekly issuance,2166,...,81.8,9650,100,2804,3.38,3.67,0.0,0.0,20100.0,930322
1,5316,1801,930711,165960,36,4610,1,46,monthly issuance,2181,...,73.5,8369,117,2618,1.79,2.31,177684.8,4167.0,229893.7,930213
2,6863,9188,930728,127080,60,2118,1,45,monthly issuance,11314,...,53.5,8390,132,2122,2.28,2.89,54873.0,7936.0,75146.0,930208
3,5325,1843,930803,105804,36,2939,1,12,monthly issuance,2235,...,58.0,8754,137,3868,3.83,4.31,86017.2,6994.0,120309.8,930130
4,7240,11013,930906,274740,60,4579,1,1,weekly issuance,13539,...,100.0,12541,167,99107,0.29,0.43,235214.0,756.0,276327.1,930214


In [299]:
# Checking if there any na or ? values in the train and test dataset

# For the train dataset
print("Nan values for train dataset: " + str(train_df.isnull().values.any()))
print('? values for train dataset: ' +  str('?' in train_df.values))

# For the test dataset
print("Nan values for train dataset: " + str(test_df.isnull().values.any()))
print('? values for train dataset: ' +  str('?' in test_df.values))

print("The only collumn that has null values is the status because that will be the attribute to predict")
print(test_df.isnull().columns[test_df.isnull().isin([True]).any()].array)


Nan values for train dataset: False
? values for train dataset: False
Nan values for train dataset: True
? values for train dataset: False
The only collumn that has null values is the status because that will be the attribute to predict
<PandasArray>
['status']
Length: 1, dtype: object


In [300]:
# Any values that are NaN in the household (avg) collumn are replaced with 0
def household_NaN(df: pd.DataFrame):
    df.loc[df['household (avg)'].isna(), 'household (avg)'] = 0
    return df    

household_NaN(train_df)
household_NaN(test_df)


,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,ratio of urban inhabitants,average salary,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '96,unemploymant rate '95,unemploymant rate '96,withdrawal total,household (avg),credit total,First trans date
0,5895,4473,970103,93960,60,1566,NaN,45,monthly issuance,5395,...,53.5,8390,132,2122,2.28,2.89,333528.0,5528.0,383076.6,951009
1,7122,10365,970104,260640,36,7240,NaN,22,monthly issuance,12756,...,52.4,8620,100,1117,1.10,1.25,264710.8,4325.0,276276.4,950902
2,6173,5724,970108,232560,48,4845,NaN,14,issuance after transaction,6922,...,74.8,10045,135,6295,1.42,1.71,612253.0,2692.0,658006.9,950419
3,6142,5591,970121,221880,60,3698,NaN,9,monthly issuance,6749,...,55.3,8899,149,2487,3.39,3.97,208837.0,323.0,247750.5,960316
4,6142,5591,970121,221880,60,3698,NaN,9,monthly issuance,6750,...,55.3,8899,149,2487,3.39,3.97,208837.0,323.0,247750.5,960316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,4989,105,981205,352704,48,7348,NaN,21,monthly issuance,127,...,67.0,9104,123,2354,1.51,2.07,225221.2,0.0,255396.6,970710
420,5221,1284,981205,52512,12,4376,NaN,55,monthly issuance,1554,...,33.9,8743,111,3894,1.88,2.43,522873.6,0.0,563908.8,970702
421,6402,6922,981206,139488,24,5812,NaN,3,weekly issuance,8356,...,41.7,8980,111,2813,1.95,2.21,652859.0,0.0,700815.6,970512
422,5346,1928,981206,55632,24,2318,NaN,70,monthly issuance,2342,...,89.9,10177,81,10108,6.63,7.75,325070.2,0.0,363519.5,970302


In [301]:
# Final balance is equal to the difference between the total amount of credit and the total amount of withdrawals
def define_final_balance(df: pd.DataFrame):
    df['final balance'] = df.apply(lambda x: x['credit total'] - x['withdrawal total'], axis=1)
    return df

train_df = define_final_balance(train_df)
test_df = define_final_balance(test_df)

In [302]:
define_final_balance(train_df)

,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,average salary,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '96,unemploymant rate '95,unemploymant rate '96,withdrawal total,household (avg),credit total,First trans date,final balance
0,5314,1787,930705,96396,12,8033,-1,30,weekly issuance,2166,...,9650,100,2804,3.38,3.67,0.0,0.0,20100.0,930322,20100.0
1,5316,1801,930711,165960,36,4610,1,46,monthly issuance,2181,...,8369,117,2618,1.79,2.31,177684.8,4167.0,229893.7,930213,52208.9
2,6863,9188,930728,127080,60,2118,1,45,monthly issuance,11314,...,8390,132,2122,2.28,2.89,54873.0,7936.0,75146.0,930208,20273.0
3,5325,1843,930803,105804,36,2939,1,12,monthly issuance,2235,...,8754,137,3868,3.83,4.31,86017.2,6994.0,120309.8,930130,34292.6
4,7240,11013,930906,274740,60,4579,1,1,weekly issuance,13539,...,12541,167,99107,0.29,0.43,235214.0,756.0,276327.1,930214,41113.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,6818,9030,961212,155616,48,3242,1,72,monthly issuance,11122,...,8994,110,9208,3.80,4.79,390706.8,2587.0,451400.8,950121,60694.0
399,5625,3189,961215,222180,60,3703,-1,29,monthly issuance,3855,...,8843,113,888,2.82,3.60,316112.6,0.0,375691.6,951129,59579.0
400,6805,8972,961221,45024,48,938,1,70,monthly issuance,11050,...,10177,81,10108,6.63,7.75,132514.2,4719.0,170898.6,960521,38384.4
401,7233,10963,961225,115812,36,3217,1,16,monthly issuance,13480,...,8427,107,1913,1.12,1.54,802426.2,0.0,844304.3,950520,41878.1


In [303]:
# Auxiliar function to deal with dates
def convert_to_datetime(date: int):
    year = math.floor(date/10000)
    month = math.floor(date/100) - year * 100 
    day = date - year * 10000 - month * 100
    date_string = str(year) + "-" + str(month) + "-" + str(day)
    date_time_object = datetime.datetime.strptime(date_string, "%y-%m-%d")
    return date_time_object

def get_num_months(start_date: int, end_date: int):
    start_datetime = convert_to_datetime(start_date)
    end_datetime = convert_to_datetime(end_date)
    return  (end_datetime.year - start_datetime.year) * 12 + (end_datetime.month - start_datetime.month)

In [304]:
# Getting the discretionary income 
# Discretionary income is the amount of an individual's income 
# that is left for spending, investing, or saving after paying 
# taxes and paying for personal necessities, such as food, shelter, 
# and clothing
def define_discretionary_income_(df: pd.DataFrame):
    df['Nº Months being a client'] = df.apply(lambda x: get_num_months(x['First trans date'], x['date']), axis=1)
    df['Discretionary income'] = df.apply(lambda x: x['final balance']/get_num_months(x['First trans date'], x['date']), axis=1)
    return df

train_df = define_discretionary_income_(train_df)
test_df = define_discretionary_income_(test_df)
print(train_df.columns)
print(train_df[['account_id', 'First trans date', 'date', 'Nº Months being a client']])


Index(['loan_id', 'account_id', 'date', 'amount', 'duration', 'payments',
       'status', 'district_id', 'frequency', 'client_id', 'type',
       'birth_number', 'no. of commited crimes '95 ', 'name ', 'region',
       'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'no. of enterpreneurs per 1000 inhabitants ',
       'no. of commited crimes '96', 'unemploymant rate '95 ',
       'unemploymant rate '96 ', 'withdrawal total', 'household (avg)',
       'credit total', 'First trans date', 'final balance',
       'Nº Months being a client', 'Discretionary income'],
      dtype='object')
     account_id  First trans date    date  Nº Months being a client
0          1787            930322  930705                    